DroidDetector: Android Malware Characterization and Detection
Using Deep Learning

**Implementation**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import glob
import time
import scipy
import shutil # for moving files from ondirectory to other
import ast
import io
import os

Deep Blief Network

In [ ]:
#Cloning Deep belief network
!git clone https://github.com/albertbup/deep-belief-network.git

Cloning into 'deep-belief-network'...
remote: Enumerating objects: 789, done.
remote: Total 789 (delta 0), reused 0 (delta 0), pack-reused 789
Receiving objects: 100% (789/789), 178.18 KiB | 333.00 KiB/s, done.
Resolving deltas: 100% (457/457), done.


In [ ]:
import os
os.chdir('deep-belief-network')
!pwd

/content/deep-belief-network


In [ ]:
!cat requirements.txt

numpy>=1.12.0
scipy>=0.18.1
scikit-learn>=0.18.1
tensorflow>=1.5.0


In [ ]:
!sudo pip install -r requirements.txt

Training

In [ ]:
import numpy as np
from numpy import load

from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

from dbn import SupervisedDBNClassification

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#Loading benign and malware matrix
Xbenign = load('/content/gdrive/My Drive/Research-Adverserial_ML/P1_DroidDetector/Data/Benign_Binary_Matrix.npy')#[:880]
Xmalware = load('/content/gdrive/My Drive/Research-Adverserial_ML/P1_DroidDetector/Data/Malware_Binary_Matrix.npy')#[:880]

print(Xbenign.shape)
print(Xmalware.shape)

(4000, 179)
(4000, 179)


In [ ]:
X = np.vstack([Xbenign, Xmalware])
print(X.shape)

(8000, 179)


In [ ]:
# making of labels(Y)
benign_Y = np.zeros(Xbenign.shape[0], dtype=np.int32)
malware_Y = np.ones(Xmalware.shape[0], dtype=np.int32)
Y = np.hstack((benign_Y, malware_Y))

In [ ]:
print(Y.shape)
print(Y)

(8000,)
[0 0 0 ... 1 1 1]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [ ]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[150, 150],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)

In [ ]:
classifier.fit(X_train, Y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 4.295236
>> Epoch 2 finished 	RBM Reconstruction error 2.756657
>> Epoch 3 finished 	RBM Reconstruction error 2.157675
>> Epoch 4 finished 	RBM Reconstruction error 1.693683
>> Epoch 5 finished 	RBM Reconstruction error 1.446827
>> Epoch 6 finished 	RBM Reconstruction error 1.353263
>> Epoch 7 finished 	RBM Reconstruction error 1.217565
>> Epoch 8 finished 	RBM Reconstruction error 1.302389
>> Epoch 9 finished 	RBM Reconstruction error 1.229201
>> Epoch 10 finished 	RBM Reconstruction error 1.192614
>> Epoch 1 finished 	RBM Reconstruction error 3.476505
>> Epoch 2 finished 	RBM Reconstruction error 2.175033
>> Epoch 3 finished 	RBM Reconstruction error 1.689672
>> Epoch 4 finished 	RBM Reconstruction error 1.502826
>> Epoch 5 finished 	RBM Reconstruction error 1.185856
>> Epoch 6 finished 	RBM Reconstruction error 1.218247
>> Epoch 7 finished 	RBM Reconstruction error 1.123205
>> Epoch 8 finished 	RBM Reconstructi

SupervisedDBNClassification()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix


In [ ]:
def different_evaluations(tn, fp, fn, tp, y_test, y_pred):
  # FPr and RNr
  FPr = fp/ (fp+tn)
  FNr = fn/ (fn+tp)
  print("False positive rate is : ", FPr)
  print("False negative rate is : ", FNr)
  # F1-score
  print("The F1-score is : ", f1_score(y_test, y_pred))

  # Recall
  print("The Recall is : ", recall_score(y_test, y_pred))

  # Precision
  print("The Precision score is : ", precision_score(y_test, y_pred))

In [ ]:
y_predict = classifier.predict(X_test)
#y_predict = predictions.reshape((predictions.shape[0]))

In [ ]:
accuracy_score(Y_test, y_predict)

0.962

In [ ]:
conf_mtrx = confusion_matrix(Y_test, y_predict)
print(conf_mtrx)

[[981  21]
 [ 55 943]]


In [ ]:
tn, fp, fn, tp = conf_mtrx.ravel()
different_evaluations(tn, fp, fn, tp, Y_test, y_predict)

False positive rate is :  0.020958083832335328
False negative rate is :  0.05511022044088176
The F1-score is :  0.9612640163098879
The Recall is :  0.9448897795591182
The Precision score is :  0.9782157676348547


**Performing Attack**

In [ ]:
X_normal_malicious = '/content/gdrive/My Drive/Research-Adverserial_ML/P1_DroidDetector/Data/AEs_with_GANs_Attack2_top2bottom/Perturbation_RatioWise/*'

In [ ]:
for npy in glob.glob(X_normal_malicious):
  y_predict = classifier.predict(load(npy))
  print(npy.split('/')[-1], accuracy_score(Y_normal_malicious, y_predict))

#print(y_predict)
#y_predict = predictions.reshape((predictions.shape[0]))

AEs_12PerCent_ratioWiese.npy 0.82
AEs_13PerCent_ratioWiese.npy 0.7666666666666667
AEs_10PerCent_ratioWiese.npy 0.8866666666666667
AEs_9PerCent_ratioWiese.npy 0.9133333333333333
AEs_11PerCent_ratioWiese.npy 0.82
AEs_8PerCent_ratioWiese.npy 0.9133333333333333
AEs_7PerCent_ratioWiese.npy 0.94
AEs_20PerCent_ratioWiese.npy 0.6066666666666667
AEs_14PerCent_ratioWiese.npy 0.7666666666666667
AEs_15PerCent_ratioWiese.npy 0.6733333333333333


In [ ]:
y_predict = classifier.predict(load(X_normal_malicious))
zeros = []
for i in range(len(y_predict)):
  if(y_predict[i]==0):
    zeros.append(i)
print(zeros)

[2, 16, 18, 21, 30, 37, 40, 44, 53, 73, 79, 129, 131, 140]


In [ ]:
Y_normal_malicious = np.ones(150, dtype=np.int32)
Y_normal_malicious[np.array(zeros)]=0

In [ ]:
accuracy_score(Y_normal_malicious, y_predict)

1.0

In [ ]:
conf_mtrx = confusion_matrix(Y_normal_malicious, y_predict)
print(conf_mtrx)

[[ 10   0]
 [  0 140]]


In [ ]:
tn, fp, fn, tp = conf_mtrx.ravel()
different_evaluations(tn, fp, fn, tp, Y_normal_malicious, y_predict)

False positive rate is :  0.0
False negative rate is :  0.0
The F1-score is :  1.0
The Recall is :  1.0
The Precision score is :  1.0
